In [7]:
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("A {word} is a")

llm = HuggingFacePipeline.from_model_id(
    model_id="openai-community/gpt2",
    task="text-generation",
    pipeline_kwargs={
        "max_new_tokens": 150,
    },
)

chain = prompt | llm

chain.invoke({"word": "potato"})

Using pad_token, but it is not set yet.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


" very delicious vegetable and it brings the flavors of the vegetables together. I've had potato bread for a long time and I know it's always delicious by the end! I'd suggest a big bowl to add it to as a snack to go along with"